### Edgar Moises Hernandez-Gonzalez
#### 21/11/19
#### Clasificacion de RWE-DWT de MI-EEG con Conv-LSTM-2D
##### Validacion cruzada de 5
##### Optimizar hiperparametros con busqueda de cuadricula

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Dropout, Flatten, Dense
from tensorflow.keras.wrappers import scikit_learn
from sklearn.model_selection import GridSearchCV

In [0]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/MI-EEG-RWE-A01T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/MI-EEG-RWE-A01E.csv", header=None)
y_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/EtiquetasA01T.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/EtiquetasA01E.csv", header=None)

In [5]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(288, 66)
(288, 66)
(288, 1)
(288, 1)


In [6]:
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

print(x.shape)
print(y.shape)

(576, 66)
(576, 1)


In [7]:
n_samples = len(y)

print("n_samples:", n_samples)

n_samples: 576


In [8]:
n_classes = len(np.unique(y_test)) #esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test

print("n_classes:", n_classes)

n_classes: 4


In [0]:
#escalar los datos para que tengan media = 0 y desviacion estandar = 1
scaler = StandardScaler().fit(x)
x = scaler.transform(x)

In [10]:
#codificar etiquetas con valores de 0 a nClases-1
le = LabelEncoder().fit(y)
y = le.transform(y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
#convertir de 2D a 5D
x = x.reshape((n_samples, 1, 1, 3, 22))

print(x.shape)

(576, 1, 1, 3, 22)


In [0]:
def model(n_filtros, p_dropout_1, n_neuronas, p_dropout_2):
    model = Sequential()
    model.add(ConvLSTM2D(n_filtros, (1,3), activation='relu', input_shape=(1,1,3,22)))
    model.add(Dropout(p_dropout_1))
    model.add(Flatten())
    model.add(Dense(n_neuronas, activation='relu'))
    model.add(Dropout(p_dropout_2))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [0]:
clf = scikit_learn.KerasClassifier(build_fn = model, epochs=20, batch_size=18, verbose=0)

In [0]:
n_filtros = [16, 32, 64, 128]
p_dropout_1 = [0.2, 0.5]
n_neuronas = [16, 32, 64]
p_dropout_2 = [0.2, 0.5]
hiperparametros = dict(n_filtros=n_filtros, p_dropout_1=p_dropout_1, n_neuronas=n_neuronas, p_dropout_2=p_dropout_2)

In [0]:
clf = GridSearchCV(clf, hiperparametros, cv=5)

In [16]:
clf.fit(x, y)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


KeyboardInterrupt: ignored

In [0]:
print(clf.best_params_)
print(clf.best_score_)

{'n_filtros': 128, 'n_neuronas': 64, 'p_dropout_1': 0.5, 'p_dropout_2': 0.2}
0.5190972242400878


In [0]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))

0.472 (+/-0.099) for {'n_filtros': 32, 'n_neuronas': 32, 'p_dropout_1': 0.2, 'p_dropout_2': 0.2}
0.443 (+/-0.125) for {'n_filtros': 32, 'n_neuronas': 32, 'p_dropout_1': 0.2, 'p_dropout_2': 0.5}
0.429 (+/-0.103) for {'n_filtros': 32, 'n_neuronas': 32, 'p_dropout_1': 0.5, 'p_dropout_2': 0.2}
0.413 (+/-0.096) for {'n_filtros': 32, 'n_neuronas': 32, 'p_dropout_1': 0.5, 'p_dropout_2': 0.5}
0.474 (+/-0.081) for {'n_filtros': 32, 'n_neuronas': 64, 'p_dropout_1': 0.2, 'p_dropout_2': 0.2}
0.444 (+/-0.109) for {'n_filtros': 32, 'n_neuronas': 64, 'p_dropout_1': 0.2, 'p_dropout_2': 0.5}
0.479 (+/-0.071) for {'n_filtros': 32, 'n_neuronas': 64, 'p_dropout_1': 0.5, 'p_dropout_2': 0.2}
0.448 (+/-0.049) for {'n_filtros': 32, 'n_neuronas': 64, 'p_dropout_1': 0.5, 'p_dropout_2': 0.5}
0.479 (+/-0.090) for {'n_filtros': 64, 'n_neuronas': 32, 'p_dropout_1': 0.2, 'p_dropout_2': 0.2}
0.490 (+/-0.111) for {'n_filtros': 64, 'n_neuronas': 32, 'p_dropout_1': 0.2, 'p_dropout_2': 0.5}
0.467 (+/-0.103) for {'n_filtr